In [ ]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import ipywidgets as widgets
import math

from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import poisson

import scipy.optimize
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split

#  ¿Es posible explicar la cantidad de billonarios en base al desarrollo país?  <a class="tocSkip"></a>

In [ ]:
df = pd.read_csv('billonarios.csv', index_col="pais")
df.head()

## Modelo 

1. Estudie y describa la distribución de Poisson en detalle. Muestre como varía la distribución en función de su parámetro $\lambda$, ¿Qué ocurre cuando $\lambda$ es pequeño? ¿Y cuando es grande?

###### La distribución de Poisson es una de las más importantes distribuciones de variable discreta. Sus principales aplicaciones se encuentra en la modelización de situaciones en las que nos interesa determinar el número de hechos  en un intervalo de tiempo, bajo supuestos de aleatoriedad y ciertas circunstancias restrictivas.

In [ ]:
@widgets.interact(lam=(0, 100 , 2))

def print_cosas(lam=0.5,):
    p = np.random.poisson(lam, 1000)
    fig, ax = plt.subplots(figsize=(12,5), tight_layout=True)
    ax.grid(color="red", which="major", axis='both', linestyle=':', linewidth=0.5)
    ax.set_title('Distribución en función de lambda = {}'.format(lam))
    ax.hist(p, bins=20);

cuando lambda toma valores pequeños, podemos ver en la gráfica que se encuentra los datos cargados hacia la izquierda, y el gráfico nos muestra una distribución de poisson, por el contrario cuando  lambda toma valores más grandes los datos siguen  una distribución Gaussiana, esto se debe al teorema central del límite.

 2. Reemplace las expresiones y obtenga una expresión analítica para el logaritmo de la verosimilitud: $\log \mathcal{L}(\theta)$. Muestre la ecuación obtenida. HINT: Puede ignorar los términos que no dependan de $\theta$. Luego obtenga una expresión analítica para la primera derivada del logaritmo de la verosimilitud. Muestra la ecuación obtenida

<img src="Img/todo.jpg">

In [ ]:
y = df["nbillonarios"].values
X = df.drop(columns=["nbillonarios"]).values

X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

## Implementación 

1. Implemente el logaritmo de la verosimilitud y su derivada usando `numpy`. Utilice operaciones vectoriales )

In [ ]:
def intensidad(thetha,X):
    f = theta[0] + np.sum(theta[1:]*X, axis=1)
    return np.exp(f)


def verosimilitud(thetha, *args):
    X, y = args
    lam = intensidad(theta, X)
    return -(np.sum(y*lam - np.exp(-lam)) - np.sum(factorial(y)))

def factorial(y):
    retorno = []
    for i in y:
        retorno.append(math.log((np.math.factorial(i)),math.exp(1)))
    return retorno

def derVerosimilitud(theta, *args):
    X, y = args
    lam = intensidad(theta, X)
    Z = np.concatenate((np.ones(shape=(len(y),1)), X), axis = 1)
    e = y - lam
    return -np.sum(e[:, np.newaxis]*Z, axis=0)

2. Implemente una rutina que encuentre el vecot de paraemtros óptimo en base a `scipy.optimize.minimize` 

In [ ]:
theta = np.random.randn(1+X.shape[1])

res = scipy.optimize.minimize(fun = verosimilitud, x0 = theta, method = 'BFGS', jac = derVerosimilitud,
                              args = (X,y), tol=1e-1)
print(res)

In [ ]:
def RCuadrado(theta, X, y, th_0):
    Ver_th_0 = -(np.sum(y*th_0 - np.exp(th_0)) - np.sum(factorial(y)))
    Ver_theta = verosimilitud(theta, X, y)
    return (Ver_th_0 - Ver_theta)/Ver_th_0

In [ ]:
r2 = RCuadrado(res.x, X, y, theta[0])
print("r2 = ", r2)

print(res.x)